# Siddhaarthan V
# MUSIC RECOMMENDATION SYSTEM
# TASK 3 BEGINNER LEVEL TASK

In [3]:
!pip install lightgbm

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [9]:
members = pd.read_csv('members.csv')

In [10]:
songs = pd.read_csv('songs.csv')

In [11]:
train = pd.read_csv('train.csv')

# data cleaning

In [12]:
ntr = 7000
nts = 3000
names=['msno','song_id','source_system_tab','source_screen_name','source_type','target']
test1 = pd.read_csv('train.csv',names=names,skiprows=ntr,nrows=nts)
test = test1.drop(['target'],axis=1)
ytr = np.array(test1['target'])
test_name = ['id','msno','song_id','source_system_tab','source_screen_name','source_type']
test['id']=np.arange(nts)
test = test[test_name]

In [13]:
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

In [14]:
members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[6:8]))
members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration_date'] = members['expiration_date'].apply(lambda x: int(str(x)[6:8]))
members = members.drop(['registration_init_time'], axis=1)

In [15]:
members_cols = members.columns
train = train.merge(members[members_cols], on='msno', how='left')
test = test.merge(members[members_cols], on='msno', how='left')

In [16]:
train = train.fillna(-1)
test = test.fillna(-1)

In [17]:
import gc
del members, songs; gc.collect();

In [18]:
cols = list(train.columns)
cols.remove('target')

In [19]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
for col in tqdm(cols):
    if train[col].dtype == 'object':
        train[col] = train[col].apply(str)
        test[col] = test[col].apply(str)

        le = LabelEncoder()
        train_vals = list(train[col].unique())
        test_vals = list(test[col].unique())
        le.fit(train_vals + test_vals)
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:28<00:00,  1.52s/it]


In [20]:
unique_songs = range(max(train['song_id'].max(), test['song_id'].max()))
song_popularity = pd.DataFrame({'song_id': unique_songs, 'popularity':0})
train_sorted = train.sort_values('song_id')
train_sorted.reset_index(drop=True, inplace=True)
test_sorted = test.sort_values('song_id')
test_sorted.reset_index(drop=True, inplace=True)

# building of a model

In [21]:
X = np.array(train.drop(['target'], axis=1))
y = train['target'].values
X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values
del train, test; gc.collect();
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state = 12)
del X, y; gc.collect();
d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid) 
watchlist = [d_train, d_valid]

In [22]:
def predict(m1_model):
    model = m1_model.fit(X_train,y_train)
    print('Training Score : {}'.format(model.score(X_train,y_train)))
    y_pred = model.predict(X_valid)
    v_test = model.predict(X_test)
    yhat = (v_test>0.5).astype(int)
    comp = (yhat==ytr).astype(int)
    acc = comp.sum()/comp.size*100
    print("Accuracy on test data for the model", acc)

In [23]:
predict(LogisticRegression())

Training Score : 0.5121212239874355
Accuracy on test data for the model 63.56666666666667


# prediction using lightgbm

In [24]:
params = {}
params['learning_rate'] = 0.4
params['application'] = 'binary'
params['max_depth'] = 15
params['num_leaves'] = 2**8
params['verbosity'] = 0
params['metric'] = 'auc'
model1 = lgb.train(params, train_set=d_train, num_boost_round=200, valid_sets=watchlist, early_stopping_rounds=10, verbose_eval=10)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.123958 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 10 rounds
[10]	training's auc: 0.711178	valid_1's auc: 0.710232
[20]	training's auc: 0.730905	valid_1's auc: 0.728782
[30]	training's auc: 0.741549	valid_1's auc: 0.738609
[40]	training's auc: 0.749137	valid_1's auc: 0.745402
[50]	training's auc: 0.756133	valid_1's auc: 0.751662
[60]	training's auc: 0.762364	valid_1's auc: 0.757353
[70]	training's auc: 0.766309	valid_1's auc: 0.760607
[80]	training's auc: 0.771415	valid_1's auc: 0.76515
[90]	training's auc: 0.775364	valid_1's auc: 0.768509
[100]	training's auc: 0.778289	valid_1's auc: 0.770738
[110]	training's auc: 0.781535	valid_1's auc: 0.773422
[120]	training's auc: 0.784836	valid_1's auc: 0.776062
[130]	training's auc: 0.78777	valid_1's auc: 0.778502
[140]	